# Homework #3 - Rossman Store Sales

Group Members:

Hoo De Lin (A0126576X)

Yap Han Chiang (A0125168E)

# Analyzing the data:

Before we try figuring out which model to fit in, we need to understand the data, and clean/modify the data to be able to feed into any model

The feature that I believe are quite important are:

- Competition
- Holidays
- Type of stores
- Type of day (Monday - Sunday)
- Having promotions
- Promo2 since

Interesting to note is that, 

In [14]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as pl
%matplotlib inline
import seaborn as sb 
from sklearn import svm
from datetime import datetime
pl.style.use('ggplot')

In [30]:
trainData = pd.read_csv('train_v2.csv', parse_dates=['Date'], dtype={'StateHoliday': 'str'})
storeData = pd.read_csv('store.csv')
# testDataUnsanitized = pd.read_csv('test_v2.csv',  parse_dates=['Date'], dtype={'StateHoliday': 'str'})

In [31]:
# Cleaning the data
def hasHoliday(row):
    if row['StateHoliday'] != "0" or row['SchoolHoliday'] != "0":
        return 1
    else:
        return 0
def calculateCompetitionAge(row):
        if math.isnan(row['CompetitionOpenSinceMonth']):
            return 0
        else:
            f_date = datetime(int(row['CompetitionOpenSinceYear']), int(row['CompetitionOpenSinceMonth']), 1, 0, 0, 0)
            return (row['Date'].to_pydatetime() - f_date).days

month_num_name_dict = {
    "1": "Jan",
    "2": "Feb",
    "3": "Mar",
    "4": "Apr",
    "5": "May",
    "6": "Jun",
    "7": "Jul",
    "8": "Aug",
    "9": "Sept",
    "10": "Oct",
    "11": "Nov",
    "12": "Dec"
}

def isHavingPromo2(row):
    if not isinstance(row['PromoInterval'], str):
        return 0
    elif month_num_name_dict[str(row['Date'].to_pydatetime().month)] in row['PromoInterval']:
        return 1
    else:
        return 0

# Possible thought: If we are not at the promo interval, should promo2Age have any effect?
def promo2Age(row):
    if row['Promo2'] == 0:
        return 0
    else:
#        https://stackoverflow.com/a/17087427
        date = str(int(row["Promo2SinceYear"])) + "-W" + str(int(row["Promo2SinceWeek"]))
        f_date = datetime.strptime(date + '-0', "%Y-W%W-%w")
        return (row['Date'].to_pydatetime() - f_date).days
trainData = pd.merge(storeData, trainData, on='Store')
trainData['HasHoliday'] = trainData.apply(lambda row: hasHoliday(row), axis=1)
trainData['CompetitionAge'] = trainData.apply(lambda row: calculateCompetitionAge(row), axis=1)
trainData['HavingPromo2'] = trainData.apply(lambda row: isHavingPromo2(row), axis=1)
trainData['Promo2Age'] = trainData.apply(lambda row: promo2Age(row), axis=1)
trainData.drop('CompetitionOpenSinceMonth', axis=1, inplace=True)
trainData.drop('CompetitionOpenSinceYear', axis=1, inplace=True)
trainData.drop('StateHoliday', axis=1, inplace=True)
trainData.drop('SchoolHoliday', axis=1, inplace=True)
trainData.drop('Promo2SinceWeek', axis=1, inplace=True)
trainData.drop('Promo2SinceYear', axis=1, inplace=True)
trainData.drop('PromoInterval', axis=1, inplace=True)
trainData.drop('Promo2', axis=1, inplace=True)
trainData.drop('Open', axis=1, inplace=True)
trainData.drop('Store', axis=1, inplace=True)

In [32]:
trainData

,StoreType,Assortment,CompetitionDistance,DayOfWeek,Date,Sales,Customers,Promo,HasHoliday,CompetitionAge,HavingPromo2,Promo2Age
0,c,a,1270.0,5,2015-01-30,5577,616,1,1,2342,0,0
1,c,a,1270.0,4,2015-01-29,5171,573,1,1,2341,0,0
2,c,a,1270.0,3,2015-01-28,4310,537,1,1,2340,0,0
3,c,a,1270.0,2,2015-01-27,4806,532,1,1,2339,0,0
4,c,a,1270.0,1,2015-01-26,4781,566,1,1,2338,0,0
5,c,a,1270.0,7,2015-01-25,0,0,0,1,2337,0,0
6,c,a,1270.0,6,2015-01-24,4840,600,0,1,2336,0,0
7,c,a,1270.0,5,2015-01-23,3586,489,0,1,2335,0,0
8,c,a,1270.0,4,2015-01-22,3492,493,0,1,2334,0,0
9,c,a,1270.0,3,2015-01-21,3299,460,0,1,2333,0,0
